# Conversational agent

In [1]:
import os
from aisModels_lc import GroqChatLLM
llm = GroqChatLLM()

Hello. How can I assist you today?
Hello! How are you today? Is there something I can help you with or would you like to chat?


In [2]:
from langchain.tools import tool

In [3]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [4]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [5]:
tools = [get_current_temperature, search_wikipedia]

In [6]:
#from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [7]:
#%%script true
functions = [format_tool_to_openai_function(f) for f in tools]
#model = ChatOpenAI(temperature=0).bind(functions=functions)
model = llm.bind(functions=functions)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

/tmp/ipykernel_31520/203605416.py:2: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  functions = [format_tool_to_openai_function(f) for f in tools]


In [8]:
result = chain.invoke({"input": "what is the weather is sf?"})

In [9]:
result.tool

'get_current_temperature'

In [10]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [11]:
from langchain.prompts import MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [12]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [13]:
result1 = chain.invoke({
    "input": "what is the weather is sf?",
    "agent_scratchpad": []
})

In [14]:
result1.tool

'get_current_temperature'

In [15]:
observation = get_current_temperature.invoke(result1.tool_input)

In [16]:
observation

'The current temperature is 18.6°C'

In [17]:
type(result1)

langchain_core.agents.AgentActionMessageLog

In [18]:
from langchain.agents.format_scratchpad import format_to_openai_functions

format_to_openai_functions([(result1, observation), ])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude": 37.7749, "longitude": -122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 315, 'total_tokens': 341, 'completion_time': 0.1042308, 'prompt_time': 0.078531683, 'queue_time': 2.128589767, 'total_time': 0.182762483}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_5c5d1b5cfb', 'finish_reason': 'function_call', 'logprobs': None}, id='run-7202c75d-0fe3-496e-9a12-3a9b62d75874-0', usage_metadata={'input_tokens': 315, 'output_tokens': 26, 'total_tokens': 341}),
 FunctionMessage(content='The current temperature is 18.6°C', name='get_current_temperature')]

In [19]:
result2 = chain.invoke({
    "input": "what is the weather is sf?", 
    "agent_scratchpad": format_to_openai_functions([(result1, observation)])
})

In [20]:
result2

AgentFinish(return_values={'output': 'The current temperature in San Francisco is 18.6°C.'}, log='The current temperature in San Francisco is 18.6°C.')

In [21]:
from langchain.schema.agent import AgentFinish

def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = chain.invoke({
            "input": user_input, 
            "agent_scratchpad": format_to_openai_functions(intermediate_steps)
        })
        
        if isinstance(result, AgentFinish):
            return result
        
        tool = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [22]:
from langchain.schema.runnable import RunnablePassthrough

agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

In [23]:
agent_chain

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_functions(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are helpful but sassy assistant')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')])
| RunnableBinding(bound=GroqChatLLM(client=<groq.resources.chat.completions.Completions object at 0x7f09e794a6e0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f09e7949b

In [24]:
from time import sleep

def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = agent_chain.invoke({
            "input": user_input, 
            "intermediate_steps": intermediate_steps
        })
        print('result1', result, '\n\n')
        if isinstance(result, AgentFinish):
            print('resultL', result, '\n\n')
            return result
        tool = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        print('tool', tool, '\n\n')
        observation = tool.run(result.tool_input)
        print('observation', observation, '\n\n')
        intermediate_steps.append((result, observation))
        print('intermediate_steps', intermediate_steps, '\n\n')
        sleep(3)

In [25]:
run_agent("what is the weather is sf?")

result1 tool='get_current_temperature' tool_input={'latitude': 37.7749, 'longitude': -122.4194} log="\nInvoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude": 37.7749, "longitude": -122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 315, 'total_tokens': 341, 'completion_time': 0.104, 'prompt_time': 0.083680039, 'queue_time': 0.004896633999999997, 'total_time': 0.187680039}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_9260b4bb2e', 'finish_reason': 'function_call', 'logprobs': None}, id='run-0c2d2f4a-03df-4069-8095-926fccf48e05-0', usage_metadata={'input_tokens': 315, 'output_tokens': 26, 'total_tokens': 341})] 


tool name='get_current_temperature' description='Fetch current temperature for given coordinates.' args_schema=<class '__main__.OpenMeteoInput'

AgentFinish(return_values={'output': 'The current temperature in San Francisco is 18.6°C.'}, log='The current temperature in San Francisco is 18.6°C.')

In [26]:
run_agent("what is langchain?")

result1 tool='search_wikipedia' tool_input={'query': 'langchain'} log="\nInvoking: `search_wikipedia` with `{'query': 'langchain'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query": "langchain"}', 'name': 'search_wikipedia'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 313, 'total_tokens': 329, 'completion_time': 0.064157386, 'prompt_time': 0.083238871, 'queue_time': 3.690296075, 'total_time': 0.147396257}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_9260b4bb2e', 'finish_reason': 'function_call', 'logprobs': None}, id='run-38e541a1-99eb-482d-85cc-1764b23165ba-0', usage_metadata={'input_tokens': 313, 'output_tokens': 16, 'total_tokens': 329})] 


tool name='search_wikipedia' description='Run Wikipedia search and get page summaries.' args_schema=<class 'pydantic.v1.main.search_wikipediaSchema'> func=<function search_wikipedia at 0x7f09e79fbeb0> 


observation Page: LangChain
Summ

AgentFinish(return_values={'output': 'LangChain is a framework designed to simplify the creation of applications using large language models (LLMs).'}, log='LangChain is a framework designed to simplify the creation of applications using large language models (LLMs).')

In [27]:
run_agent("hi!")

result1 return_values={'output': 'What can I help you with today?'} log='What can I help you with today?' 


resultL return_values={'output': 'What can I help you with today?'} log='What can I help you with today?' 




AgentFinish(return_values={'output': 'What can I help you with today?'}, log='What can I help you with today?')

In [28]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

In [29]:
agent_executor.invoke({"input": "what is langchain?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'langchain'}`


Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: DataStax
Summary: DataStax, Inc. is a real-time data for AI company based in Santa Clara, California. Its product Astra DB is a cloud database-as-a-service based on Apache Cassandra. DataStax also offers DataStax Enterprise (DSE), an on-premises database built on Apache Cassandra, and Astra Streaming, a messaging and event streaming cloud service based on Apache Pulsar. As of June 2022, the company has roughly 800 customers distributed in over 50 countries.



Page: Retrieval-augmented generation
Summary: Retrieval augmented generation (RAG) is a typ

{'input': 'what is langchain?',
 'output': 'LangChain is a framework designed to simplify the creation of applications using large language models (LLMs).'}

In [30]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
That's nice, Bob. How can I assist you today?

> Finished chain.


{'input': 'my name is bob',
 'output': "That's nice, Bob. How can I assist you today?"}

In [31]:
agent_executor.invoke({"input": "what is my name"})



> Entering new AgentExecutor chain...
I don't have that information. You haven't told me your name.

> Finished chain.


{'input': 'what is my name',
 'output': "I don't have that information. You haven't told me your name."}

In [32]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [33]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [34]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

In [35]:
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [36]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
That's nice, Bob. How can I assist you today?

> Finished chain.


{'input': 'my name is bob',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content="That's nice, Bob. How can I assist you today?")],
 'output': "That's nice, Bob. How can I assist you today?"}

In [37]:
agent_executor.invoke({"input": "whats my name"})



> Entering new AgentExecutor chain...
You told me earlier, Bob. Your name is Bob.

> Finished chain.


{'input': 'whats my name',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content="That's nice, Bob. How can I assist you today?"),
  HumanMessage(content='whats my name'),
  AIMessage(content='You told me earlier, Bob. Your name is Bob.')],
 'output': 'You told me earlier, Bob. Your name is Bob.'}

In [38]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`
responded: I'd be happy to help you with that, Bob. However, I need to make a function call to get the current temperature in San Francisco. I'll assume the coordinates for San Francisco are latitude: 37.7749 and longitude: -122.4194.



The current temperature is 18.6°CThe current temperature in San Francisco is 18.6°C.

> Finished chain.


{'input': 'whats the weather in sf?',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content="That's nice, Bob. How can I assist you today?"),
  HumanMessage(content='whats my name'),
  AIMessage(content='You told me earlier, Bob. Your name is Bob.'),
  HumanMessage(content='whats the weather in sf?'),
  AIMessage(content='The current temperature in San Francisco is 18.6°C.')],
 'output': 'The current temperature in San Francisco is 18.6°C.'}

### Create a chatbot

In [39]:
@tool
def create_your_own(query: str) -> str:
    """This function can do whatever you would like once you fill it in """
    print(type(query))
    return query[::-1]

In [40]:
tools = [get_current_temperature, search_wikipedia, create_your_own]

In [41]:
import panel as pn  # GUI
pn.extension()
import panel as pn
import param

class cbfs(param.Parameterized):
    
    def __init__(self, tools, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.functions = [format_tool_to_openai_function(f) for f in tools]
        #self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)
        self.model = llm.bind(functions=self.functions)
        
        self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful but sassy assistant"),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)
    
    def convchain(self, query):
        
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        
        self.answer = result['output'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)


    def clr_history(self,count=0):
        self.chat_history = []
        return 

In [42]:
cb = cbfs(tools)

inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp) 

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=400),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# QnA_Bot')),
    pn.Tabs(('Conversation', tab1))
)
dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'20743fa2-05e0-45ac-a172-901fb8e28ac2': {'version…